<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335206/logos/Logo_des_Forschungszentrums_J_C3_BClich_seit_2018_hcliq4.svg"  width=250 align='left' style="margin-top:30px"/>
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637333672/logos/ebrains_logo_tndl7p.svg" width="280" align='left' style="margin-left:50px; margin-top:25px">
</div>  

<br><br><br><br>
<br>

# Simulation of  dose-response curves of agonists using kinetic values

In this tutorial we will simulate mathematical model of a signaling pathway to obtain dose-response curves, and consequently, predict the *efficacy (EC$_{50}$)* of drugs. 

## <span style="color:red">This tutorial just works specifically on EBRAINS</span>

## Install SSB toolkit on EBRAINS
#### If you didn't clone the repository into ebrains run the following code

In [ ]:
#@title Install dependencies
%%bash

# download SSB source code and install dependencies
if [ ! -d "SSBtoolkit/" ]; then
  git clone https://github.com/rribeiro-sci/SSBtoolkit.git --quiet
  pip install -r SSBtoolkit/requirements.txt
fi


#### If you have cloned the repository run the following code

In [ ]:
from Ipython.display import clear_output
!pip install -r requirements.txt
clear_ouput()

### Import Dependencies

In [ ]:
#Import Python dependencies
import os, sys, json, math, site
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

from src.lib.ssbtoolkit import convert, get, simulation

In [ ]:
#Setting BioNetGen environment path
distpath = '/opt/app-root/src/.local/lib/python3.8/site-packages/'
BioNetGen=os.path.join(distpath, 'bionetgen/bng-linux:')
mypath=%env PATH
newpath=BioNetGen+mypath
%env PATH=$newpath

In [ ]:
#Test bioservices
from bioservices import UniProt
u = UniProt(verbose=False)

## Load experimental Data

Once the SSBtoolkit environment is set up we are ready to start to simulate.

We will begin by loading the kinetic data of some ligands of the Adenosine 2A receptor. This data was taken from *([Guo, D. et al., 2017](https://pubs.acs.org/doi/10.1021/acs.chemrev.6b00025))*. This data can be found in `examples/A2AR_Kinetic_values_example.csv`

In [ ]:
data = pd.read_csv('example/A2AR_Kinetic_values_example.csv')
data

Commonly the kinetic parameters, k<sub>on</sub> and k<sub>off</sub>, are measured experimentally at low temperatures in order to slow down the reaction, so it can be measured. Therefore, we will use the SSBtoolkit built-in function `convert.KineticTempScale()` to scale the kinetic parameters to room temperature (25 °C). This is achieved using the free energy equation:

$ \Delta G = -RTln(K_d)$ 

$K_d = \frac{k_{off}}{k_{on}}$

In [ ]:
#We will create a new dataframe with the scaled values 
data_scaled = data.copy()
data_scaled['kon (1/(uM*s))'] = data.apply(lambda row: convert.KineticTempScale(row['kon (1/(uM*s))'],row['koff (1/s)'], row['T (°C)'], 25, Tu='C')[0], axis=1)
data_scaled['koff (1/s)'] = data.apply(lambda row: convert.KineticTempScale(row['kon (1/(uM*s))'],row['koff (1/s)'], row['T (°C)'], 25, Tu='C')[1], axis=1)
data_scaled['T (°C)'] = 25
data_scaled

## Selection of the signaling pathway 

The core of the SSB framework is, naturally, the mathematical models of the GPCRs' signaling pathways. 

Since G-protein sub-families are classified by their $\alpha$ subunits, this classfication as been served to identify their signaling pathways:
* G<sub>s</sub>
* G<sub>i/o</sub>
* G<sub>q/11</sub>
* G<sub>12/13</sub>

📖 See [The SSB toolkit](https://github.com/rribeiro-sci/SSBtoolkit/blob/main/docs/ssb_toolkit.md) documentation for more details.

We can define manualy the G$\alpha$ pathway we want to work with, or simply query our internal database of human GPCR pathways using the UNIPROT id of our receptor using the SSBtoolkit built-in function `get.gprotein()`. The UNIPROT id for the human Adenosine A2 receptot is [P29274](https://www.uniprot.org/uniprot/P29274).

In [ ]:
uniprotID = 'P29274'
gprotein=get.gprotein(uniprotID)
gprotein

<span style="color:black">⚠️ WARNING: our toolkit was specifically design for human GPCRs. The quering for pathways for GPCRs other than Human may be included in the future. However, if you want to study a non-human GPCR you can still use the SSB toolkit by setting manually the G$\alpha$ pathway.</span>

##  Preparation, Simulation and Analysis

To obtain a dose-response curve from the simulation of signaling pathways, individual simulations of the pathway according to a series of ligand concentrations must be performed (as it would be done in the wet-lab).  

To define an array of ligand concentrations we will use a geometric progression. The reason why we use a geometric progression is due to the impact of the dilution fraction on the accuracy of K<sub>d</sub> and EC<sub>50</sub>/IC<sub>50</sub> values experimentally estimated. This factor, that defines the spacing between the adjacent concentration values, has an impact on the concentration values that are on the linear portion of the curve. Therefore, using a geometric progression we can mimic the experimental conditions where each concentration equals to the power of 2 of the previous lowest concentration *([Sebaugh, J.L., 2011](https://doi.org/10.1002/pst.426))*

<span style="color:black"> ⚠️ WARNING: the SSB toolkit uses μM as default concentration units.</span>


In [ ]:
# setting the range of ligand concentration
lig_conc_min = 1E-4 # μM
lig_conc_max = 1E4  # μM
lig_conc_range = np.geomspace(lig_conc_min, lig_conc_max, 20) # 20 concentration values

# Setting receptor concentration
receptor_conc = 1E-3 #μM

# defining other simulation parameters:
time = 10000  # time of simulation in seconds
nsteps = 1000   # number of time steps

## Integration of ODEs 

After having defined all the simulation parameters we are ready to proceed with the simulations. A simulation of a methamatical model of a signaling pathway consists of the integration of a set of ordinary differential equations (ODEs) as function of time. Since each ODE represents a molecular event of the signaling pathway, when integrated over time, such equations can describe how the concentration of species inside the model changes over time.

In the <b>Tutorial1</b>, the key point is using  the drug-receptor affinity value (K<sub>d</sub>) to fire up the model. However, in this tutorial we will use the kinetic parameters to simulate the ligand-receptor binding.

Because we are using kinetic values we have to set `kinetics=True` in the `simulation.activation.SetSimulationParameters()` instance. If we use option `kinetcs=True`, we also need to pass to the instance a dictionary of parameters. We can see a list of all parameters that can be changed for each pathway [here](https://github.com/rribeiro-sci/SSBtoolkit/tree/main/SI).

For this tutorial we just need to include the following kinetic parameters:
* `RL_kon` for the ligand-receptor interaction forward parameter (μM<sup>-1</sup>s<sup>-1</sup>)
* `RL_koff` for the ligand-receptor interaction reverse parameter (s<sup>-1</sup>)

In [ ]:
#preparing the parameters for each ligand:
kinetic_parameters=[]
for k, v in data_scaled.iterrows():
    kinetic_parameters.append({'RL_kon':v['kon (1/(uM*s))'],'RL_koff':v['koff (1/s)']})

In [ ]:
#creating a simulation instance
sim = simulation.activation()

In [ ]:
#Setting the simulation parameters
sim.SetSimulationParameters(ligands=data_scaled.cmpd.to_list()[0:2], 
                            pathway=gprotein, 
                            receptor_conc=receptor_conc, 
                            lig_conc_range=lig_conc_range, 
                            ttotal=time, 
                            nsteps=nsteps,  
                            binding_kinetics=True, 
                            binding_kinetic_parameters=kinetic_parameters)

In [ ]:
#Start the simulation
sim.Run()

In the end, the concentration values of the species of the signaling pathway over the simulation time will be saved inside the instance.

The response of a signaling pathway is, naturally, represented by the increase or decrease of one of the species described by the model. So, to predict the dose-response curve we need, firstly, to extract the maximum concentration value orbserved for one specie from each individual simulation (from the series of simulations for each ligand concentration). Then, such values will be fitted to a logistic regression. 

To achieve this, we will use the analysis function:

In [ ]:
sim.Analysis()

We can now to plot the dose-response curves:

In [ ]:
sim.Curve()

Finnaly, from the dose-response curves we can interpolate the EC<sub>50</sub> values.


In [ ]:
sim.Potency()

💡 The potency predicted values can be exported as a python dictionary using the function `sim.PotencyToDict()` or saved in a csv file: `sim.PotencyToCSV()`. 

## Save results on Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
sim.PotencyToCSV(path='/gdrive/MyDrive/XX') ## change XX accordingly.

## Congratulations! 

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with SSBtoolkit, we encourage you to finish the rest of the tutorials in this series. 

## Cite Us

If you use or adapt this tutorial for your own research projects please cite us.

```
@article{XXX,
    title={XXX},
    author={XXX},
    publisher={XXX},
    note={\url{XXX}},
    year={XXX}
}
```



## Acknowledgments

EU Human Brain Project (SGA1 and SGA2): This open source software was developed in part in the Human Brain Project funded from the European Union's Horizon 2020 Framework Programme for Research and Innovation under Specific Grant Agreements No 720270 and No. 78907 (Human Brain Project SGA1 and SGA2).

<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
    <img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1642677502/logos/COFUNDED_EU_j2ktlp.jpg" width="300" align='left' style="margin-left:50px">
</div>  